# Amőba program háttere

Egy segédprogram, amin keresztül ki tudunk próbálni különböző "gépi ellenfeleket"!
Ha egyáltalán nem ismerős a programozás, ne aggódj! A lényeg a képeken fog látszani.

---

## A kódrészekben fontosabb szavak, amikkel találkozhatsz:

* `Tabla`: a "tábla" állapotát fogjuk így hívni
* `JatekEredmeny`: A lehetséges eredmények gyűjtője: `BEFEJEZETLEN`, `DONTETLEN`, `X_NYERT`, vagy `O_NYERT`
* `X`, `O`: a táblán helyezhető jelölések, X vagy O. A nem kitöltött mezőkre URES néven is hivatkozunk időnként.
* mutasd_meg(tabla): kirajzolja a játékteret, hogy követni tudjuk
* tabla.lepj(): egy lépés végrehajtása (X vavgy O elhelyezése)


In [1]:
from IPython.display import HTML, display, clear_output
from tic_tac_toe.Tabla import Tabla, JatekEredmeny, X, O
import numpy as np
import pandas as pd
import time


def mutasd_meg(tabla):
  clear_output(wait=True)
  print(pd.DataFrame(np.array(tabla.state_to_charlist())))

egy új játékterületet létre tudunk hozni:

In [2]:
tabla = Tabla()
mutasd_meg(tabla)

   0  1  2
0         
1         
2         


Kód: A táblán lépj egy véletlenszerű üres mezőre X-szel!

In [3]:
tabla = Tabla()
tabla.lepj(tabla.veletlen_ures_cella(), X)
mutasd_meg(tabla)

   0  1  2
0  x      
1         
2         


Bővítsük ki: ne egyet lépjünk, hanem addig felváltva véletlenszerűen, amíg vége nincs a játéknak! 

Először is újrakezdjük a játékot (tabla.torles() ), utána...

In [4]:
varakoztatas = 1

In [5]:
tabla.torles()
vege = False
while not vege:
   mutasd_meg(tabla)
   time.sleep(varakoztatas)
   _, eredmeny, vege = tabla.lepj(tabla.veletlen_ures_cella(), X)

   
   mutasd_meg(tabla)
   if vege:
       if eredmeny == JatekEredmeny.DONTETLEN:
           print("Döntetlen!")
       else:
           print("X nyert!")
   else:
       time.sleep(varakoztatas)
       _, eredmeny, vege = tabla.lepj(tabla.veletlen_ures_cella(), O)
       mutasd_meg(tabla)
       if vege:
            if eredmeny == JatekEredmeny.DONTETLEN:
               print("Döntetlen")
            else:
               print("O nyert!")


   0  1  2
0  x  o  x
1  o  x  o
2        x
X nyert!


ezt a programot elnevezzük "play_game"-nek:

In [6]:
from tic_tac_toe.Jatekos import Jatekos


def play_game(tabla: Tabla, player1: Jatekos, player2: Jatekos):
    player1.uj_jatek(X)
    player2.uj_jatek(O)
    tabla.torles()
    
    vege = False
    while not vege:
        eredmeny, vege = player1.lepj(tabla)
        if vege:
            if eredmeny == JatekEredmeny.DONTETLEN:
                vegeredmeny = JatekEredmeny.DONTETLEN
            else:
                vegeredmeny =  JatekEredmeny.X_NYERT
        else:
            eredmeny, vege = player2.lepj(tabla)
            if vege:
                if eredmeny == JatekEredmeny.DONTETLEN:
                    vegeredmeny =  JatekEredmeny.DONTETLEN
                else:
                    vegeredmeny =  JatekEredmeny.O_NYERT
        
    player1.vegeredmeny(vegeredmeny)
    player2.vegeredmeny(vegeredmeny)
    return vegeredmeny


---

# Az első számítógépes játékosunk: Véletlent Lépő Robot

In [9]:
from tic_tac_toe.VeletlentLepoRobot import VeletlentLepoRobot

player1 = VeletlentLepoRobot()
player2 = VeletlentLepoRobot()
eredmeny = play_game(tabla, player1, player2)
mutasd_meg(tabla)
if eredmeny == JatekEredmeny.X_NYERT:
    print("X nyert")
elif eredmeny == JatekEredmeny.O_NYERT:
    print("O nyert")
else:
    print("Döntetlen")

   0  1  2
0  x  x  o
1  o  o  x
2  x  o  x
Döntetlen


---

# Véletlent Lépő Robot teljesítménye

Sok játék után, ha mindkét játékos véletlenszerűen lép, ki szokott nyerni általában?

In [15]:
from ipywidgets import IntProgress
from IPython.display import display

jatekok_szama = 1000

dontetlenek_szama = 0
x_nyert = 0
o_nyert = 0

egyik_jatekos = VeletlentLepoRobot()
masik_jatekos = VeletlentLepoRobot()

f = IntProgress(min=0, max=jatekok_szama)
display(f)

for _ in range(jatekok_szama):
    eredmeny = play_game(tabla, egyik_jatekos, masik_jatekos)
    if eredmeny == JatekEredmeny.X_NYERT:
        x_nyert += 1
    elif eredmeny == JatekEredmeny.O_NYERT:
        o_nyert += 1
    else:
        dontetlenek_szama += 1
    f.value += 1
        
print("{} játszma után döntetlen: {}, X nyert: {} és O nyert: {}.".format(jatekok_szama, dontetlenek_szama, 
                                                                        x_nyert, o_nyert))

print("Arányokban kifejezve döntetlen: {:.2%}, X nyert: {:.2%} O nyert: {:.2%}".format(
    dontetlenek_szama / jatekok_szama, x_nyert / jatekok_szama, o_nyert / jatekok_szama))

IntProgress(value=0, max=1000)

1000 játszma után döntetlen: 128, X nyert: 563 és O nyert: 309.
Arányokban kifejezve döntetlen: 12.80% X nyert: 56.30% O nyert: 30.90%


---

## Összefoglalva

Tudjuk, hogy ha mindkét fél mindig tökéleteset lép, mindig döntetlen az eredmény.

| Player | P1 Win | P2 Win | Draw |
| --- | ---| --- | --- |
| Tökéletes | 0%     | 0%     | 100% |
| Véletlen lépések  | ?    | ? | ? |

## Kérdés

* Mennyire "Mesterséges Intellegencia" ez a játékos?